# Azure Weather Agent Demo with Real Weather API

This notebook demonstrates an AI agent that provides **real weather information** using the OpenWeatherMap API.

## Features
- 🌤️ **Real Weather Data** - Uses OpenWeatherMap API for current conditions
- 🤖 **AI Agent** - Powered by Azure OpenAI
- 🔧 **Function Calling** - Agent automatically calls weather tools when needed
- 💬 **Interactive Chat** - Natural conversation about weather

## Prerequisites
- Azure OpenAI endpoint and API key configured in `.env`
- OpenWeatherMap API key (free from https://openweathermap.org/api)
- Environment variables properly loaded

## Step 1: Setup and Load Environment Variables

First, let's load all required environment variables from the `.env` file.

In [1]:
import os
import sys
from pathlib import Path

# Detect if running in Jupyter and adjust path accordingly
try:
    import ipykernel
    # Running in notebook
    in_notebook = True
except ImportError:
    in_notebook = False

from dotenv import load_dotenv

# Load environment variables from .env file
if '__file__' in globals():
    # Running as a script
    env_path = Path(__file__).resolve().parents[4] / ".env"
else:
    # In notebook - get the notebook's directory
    # The notebook is in: python/packages/devui/samples/weather_agent_azure/
    # We need to get to: python/.env
    notebook_dir = Path.cwd()
    
    # Check if we're already in the weather_agent_azure directory
    if 'weather_agent_azure' in str(notebook_dir):
        # Go up 4 levels: weather_agent_azure -> samples -> devui -> packages -> python
        env_path = notebook_dir.parents[3] / ".env"
    else:
        # Fall back: try to construct path from repository root
        # Navigate to python directory and find .env
        parts = notebook_dir.parts
        if 'agent-framework' in parts:
            framework_idx = parts.index('agent-framework')
            repo_root = Path(*parts[:framework_idx+1])
            env_path = repo_root / "python" / ".env"
        else:
            # Last resort: use cwd and go up to find python dir
            env_path = notebook_dir / "python" / ".env"

load_dotenv(dotenv_path=env_path, override=True)

print("🔧 Environment Configuration:")
print(f"   Current directory: {Path.cwd()}")
print(f"   .env path: {env_path}")
print(f"   .env exists: {env_path.exists()}\n")

# Verify Azure environment variables
azure_vars = [
    'AZURE_OPENAI_ENDPOINT',
    'AZURE_OPENAI_API_KEY',
    'AZURE_OPENAI_CHAT_DEPLOYMENT_NAME'
]

print("☁️  Azure OpenAI Configuration:")
for var in azure_vars:
    value = os.getenv(var)
    if value:
        # Show partial value for security
        display_value = value[:20] + '...' if len(value) > 20 else value
        print(f"   ✓ {var}: {display_value}")
    else:
        print(f"   ✗ {var}: NOT SET")

# Verify OpenWeatherMap API key
weather_key = os.getenv('OPENWEATHERMAP_API_KEY')
print("\n🌤️  Weather API Configuration:")
if weather_key:
    print(f"   ✓ OPENWEATHERMAP_API_KEY: {weather_key[:10]}...")
else:
    print(f"   ✗ OPENWEATHERMAP_API_KEY: NOT SET")
    print(f"   ⚠️  Get free API key at: https://openweathermap.org/api")

print("\n✅ Environment setup complete!")

🔧 Environment Configuration:
   Current directory: /Users/arturoquiroga/GITHUB/agent-framework/python/packages/devui/samples/weather_agent_azure
   .env path: /Users/arturoquiroga/GITHUB/agent-framework/python/.env
   .env exists: True

☁️  Azure OpenAI Configuration:
   ✓ AZURE_OPENAI_ENDPOINT: https://aq-ai-foundr...
   ✓ AZURE_OPENAI_API_KEY: Aq8FrmU2q7vONgcienmy...
   ✓ AZURE_OPENAI_CHAT_DEPLOYMENT_NAME: gpt-4.1

🌤️  Weather API Configuration:
   ✓ OPENWEATHERMAP_API_KEY: e893fc646f...

✅ Environment setup complete!


## Step 2: Import Real Weather Functions

Import the real weather utility functions from the `azure_ai_real_weather` module.

In [2]:
# Add the real weather module to Python path
notebook_dir = Path.cwd()

# Build path to weather_utils
if 'weather_agent_azure' in str(notebook_dir):
    # Go up 4 levels to get to python dir, then navigate to weather utils
    python_dir = notebook_dir.parents[3]
    weather_utils_path = python_dir / "samples" / "getting_started" / "agents" / "azure_ai_real_weather"
else:
    # Try to find from repo root
    parts = notebook_dir.parts
    if 'agent-framework' in parts:
        framework_idx = parts.index('agent-framework')
        repo_root = Path(*parts[:framework_idx+1])
        weather_utils_path = repo_root / "python" / "samples" / "getting_started" / "agents" / "azure_ai_real_weather"
    else:
        # Last resort
        weather_utils_path = notebook_dir / "python" / "samples" / "getting_started" / "agents" / "azure_ai_real_weather"

if str(weather_utils_path) not in sys.path:
    sys.path.insert(0, str(weather_utils_path))

print(f"📦 Adding weather utilities from: {weather_utils_path}")
print(f"   Path exists: {weather_utils_path.exists()}")

# Import the real weather functions
try:
    from weather_utils import get_real_weather, get_real_weather_detailed
    print("\n✅ Successfully imported real weather functions:")
    print("   - get_real_weather()")
    print("   - get_real_weather_detailed()")
except ImportError as e:
    print(f"\n❌ Error importing weather utilities: {e}")
    print("   Make sure the weather_utils.py file exists at the expected location.")
    print(f"   Looking at: {weather_utils_path}")

📦 Adding weather utilities from: /Users/arturoquiroga/GITHUB/agent-framework/python/samples/getting_started/agents/azure_ai_real_weather
   Path exists: True

✅ Successfully imported real weather functions:
   - get_real_weather()
   - get_real_weather_detailed()


## Step 3: Test Weather Functions

Let's test the weather functions directly to ensure they work properly.

In [3]:
import asyncio

# Test the real weather function
print("🧪 Testing real weather API...\n")

test_city = "Mexico City"
print(f"Getting weather for {test_city}...\n")

weather_result = await get_real_weather(test_city)
print(weather_result)

print("\n" + "="*60 + "\n")

# Test another city
test_city2 = "Toronto"
print(f"Getting weather for {test_city2}...\n")

weather_result2 = await get_real_weather(test_city2)
print(weather_result2)

🧪 Testing real weather API...

Getting weather for Mexico City...

Weather in Mexico City:
- Conditions: scattered clouds
- Temperature: 21.64°C (feels like 21.06°C)
- Humidity: 46%
- Wind Speed: 2.06 m/s


Getting weather for Toronto...

Weather in Toronto:
- Conditions: clear sky
- Temperature: 21.09°C (feels like 20.85°C)
- Humidity: 61%
- Wind Speed: 8.05 m/s
Weather in Mexico City:
- Conditions: scattered clouds
- Temperature: 21.64°C (feels like 21.06°C)
- Humidity: 46%
- Wind Speed: 2.06 m/s


Getting weather for Toronto...

Weather in Toronto:
- Conditions: clear sky
- Temperature: 21.09°C (feels like 20.85°C)
- Humidity: 61%
- Wind Speed: 8.05 m/s


## Step 4: Create Azure Weather Agent

Now let's create an AI agent that uses these real weather functions as tools.

In [4]:
from agent_framework import ChatAgent
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Create Azure OpenAI chat client
print("🤖 Creating Azure OpenAI chat client...")
chat_client = AzureOpenAIChatClient(
    credential=AzureCliCredential(),
)
print("   ✓ Chat client created\n")

# Create the weather agent with real weather tools
print("🌤️  Creating Weather Agent...")
weather_agent = ChatAgent(
    name="RealWeatherAgent",
    description="An intelligent weather assistant that provides real-time weather information from OpenWeatherMap API",
    instructions="""You are a helpful weather assistant with access to real-time weather data.
    
    When users ask about weather:
    - Use get_real_weather() for current weather conditions
    - Use get_real_weather_detailed() for more comprehensive information
    - Be conversational and friendly
    - Interpret the data and provide helpful insights
    - If users mention travel or outdoor activities, proactively suggest checking weather for those locations
    
    Always call the weather functions when asked about weather - don't make up weather data.
    """,
    chat_client=chat_client,
    tools=[get_real_weather, get_real_weather_detailed],
)

print("   ✓ Weather Agent created successfully!")
print(f"   Agent name: {weather_agent.name}")
print(f"   Agent description: {weather_agent.description}")
print(f"   Available tools:")
print("      - get_real_weather()")
print("      - get_real_weather_detailed()")

🤖 Creating Azure OpenAI chat client...
   ✓ Chat client created

🌤️  Creating Weather Agent...
   ✓ Weather Agent created successfully!
   Agent name: RealWeatherAgent
   Agent description: An intelligent weather assistant that provides real-time weather information from OpenWeatherMap API
   Available tools:
      - get_real_weather()
      - get_real_weather_detailed()


## Step 5: Chat with the Weather Agent

Now let's have a conversation with the agent! It will automatically call the weather APIs when needed.

### Example 1: Simple Weather Query

In [5]:
# Simple weather query
user_message = "What's the weather like in Paris right now?"

print(f"💬 User: {user_message}\n")
print("🤖 Agent: ", end="")

response = await weather_agent.run(user_message)

print(response.text)

💬 User: What's the weather like in Paris right now?

🤖 Agent: Right now in Paris, it's a clear sky with temperatures at 13.4°C (feels like 12.6°C). Humidity is fairly high at 71%, and there's a gentle breeze with wind speeds around 2 m/s. It's a pleasant evening—perfect for a walk or some outdoor dining! 

Let me know if you want a more detailed forecast or tips for activities.
Right now in Paris, it's a clear sky with temperatures at 13.4°C (feels like 12.6°C). Humidity is fairly high at 71%, and there's a gentle breeze with wind speeds around 2 m/s. It's a pleasant evening—perfect for a walk or some outdoor dining! 

Let me know if you want a more detailed forecast or tips for activities.


### Example 2: Multi-City Comparison

In [6]:
# Multi-city comparison
user_message = "I'm planning a trip and trying to decide between Tokyo and Singapore. Can you compare the weather in both cities?"

print(f"💬 User: {user_message}\n")
print("🤖 Agent: ", end="")

response = await weather_agent.run(user_message)

print(response.text)

💬 User: I'm planning a trip and trying to decide between Tokyo and Singapore. Can you compare the weather in both cities?

🤖 Agent: Here's a side-by-side look at the current weather in Tokyo and Singapore:

Tokyo
- Weather: Light intensity shower rain
- Temperature: 21.6°C (feels like 22.1°C)
- Humidity: 86%
- Wind: 3.6 m/s
- Cloudiness: 75%

Singapore
- Weather: Broken clouds (no current rain)
- Temperature: 28.1°C (feels like 32.2°C)
- Humidity: 79%
- Wind: 1.5 m/s
- Cloudiness: 75%

What does this mean for your trip?
- Tokyo is cooler and currently experiencing showers, so you may want to pack an umbrella or rain jacket.
- Singapore is much warmer and more humid, with “feels like” temperatures over 32°C, though there’s no rain right now.

If you prefer cooler weather and don't mind a bit of rain, Tokyo might be more comfortable. If you enjoy warmth and higher humidity, Singapore is your spot! Let me know if you need more details or want to check the forecast for your trip dates.
Her

### Example 3: Travel Advice

In [7]:
# Travel advice based on weather
user_message = "What's the weather in Barcelona? Should I bring a jacket?"

print(f"💬 User: {user_message}\n")
print("🤖 Agent: ", end="")

response = await weather_agent.run(user_message)

print(response.text)

💬 User: What's the weather in Barcelona? Should I bring a jacket?

🤖 Agent: Right now in Barcelona, it's scattered clouds with a temperature of around 20.9°C (feels like 20.7°C) and mild humidity. The wind is light.

You probably won't need a heavy jacket—it's quite pleasant! If you tend to feel chilly with just a light breeze or in the evening, a light jacket or sweater would be enough. Otherwise, you should be comfortable without one. Enjoy Barcelona!
Right now in Barcelona, it's scattered clouds with a temperature of around 20.9°C (feels like 20.7°C) and mild humidity. The wind is light.

You probably won't need a heavy jacket—it's quite pleasant! If you tend to feel chilly with just a light breeze or in the evening, a light jacket or sweater would be enough. Otherwise, you should be comfortable without one. Enjoy Barcelona!


### Example 4: Custom Query

Try your own weather query!

In [8]:
# Your custom query - edit the message below!
user_message = "What's the weather like in New York City?"

print(f"💬 User: {user_message}\n")
print("🤖 Agent: ", end="")

response = await weather_agent.run(user_message)

print(response.text)

💬 User: What's the weather like in New York City?

🤖 Agent: Right now in New York City, the weather is clear with a temperature of about 26.6°C. It feels quite pleasant, with moderate humidity at 41% and a gentle breeze. It's a great day to be outdoors! If you have any plans or need a detailed forecast, just let me know.
Right now in New York City, the weather is clear with a temperature of about 26.6°C. It feels quite pleasant, with moderate humidity at 41% and a gentle breeze. It's a great day to be outdoors! If you have any plans or need a detailed forecast, just let me know.


## Step 6: Inspect Agent Run Details

Let's look at the details of the agent's execution to see how it used the weather tools.

In [9]:
# Show detailed information about the last response
print("📊 Agent Run Details:\n")

# Show available attributes
if hasattr(response, 'status'):
    print(f"Status: {response.status}")

if hasattr(response, 'text'):
    print(f"\nResponse Text:\n{response.text}")

# Try to show run_id if available
if hasattr(response, 'run_id'):
    print(f"\nRun ID: {response.run_id}")

# Check what attributes the response actually has
print(f"\n📋 Response Object Info:")
print(f"   Type: {type(response).__name__}")
available_attrs = [attr for attr in dir(response) if not attr.startswith('_')]
print(f"   Available attributes: {', '.join(available_attrs[:10])}...")

# Note about tools - they're called internally but may not be directly visible
print(f"\n💡 Note: The agent called weather API functions internally to generate this response.")
print(f"   The actual API calls are handled by the agent framework automatically.")

📊 Agent Run Details:


Response Text:
Right now in New York City, the weather is clear with a temperature of about 26.6°C. It feels quite pleasant, with moderate humidity at 41% and a gentle breeze. It's a great day to be outdoors! If you have any plans or need a detailed forecast, just let me know.

📋 Response Object Info:
   Type: AgentRunResponse
   Available attributes: additional_properties, construct, copy, created_at, dict, from_agent_response_generator, from_agent_run_response_updates, from_orm, json, messages...

💡 Note: The agent called weather API functions internally to generate this response.
   The actual API calls are handled by the agent framework automatically.


## Step 7: Conversational Context (Multi-Turn)

The agent maintains conversation context. Let's have a multi-turn conversation.

In [10]:
# Start a new conversation thread
print("💬 Starting a multi-turn conversation...\n")
print("="*60)

# Turn 1
message1 = "What's the weather in Sydney?"
print(f"\n👤 You: {message1}\n")
response1 = await weather_agent.run(message1)
print(f"🤖 Agent: {response1.text}")

print("\n" + "="*60)

# Turn 2 - reference to previous context
message2 = "How does that compare to Melbourne?"
print(f"\n👤 You: {message2}\n")
response2 = await weather_agent.run(message2)
print(f"🤖 Agent: {response2.text}")

print("\n" + "="*60)

💬 Starting a multi-turn conversation...


👤 You: What's the weather in Sydney?

🤖 Agent: Right now in Sydney, the weather is clear with no clouds in the sky! The temperature is 18.6°C, feeling a little cooler at 17.9°C. Humidity is at a comfortable 51%, and there’s a gentle breeze with wind speeds of 6.2 m/s. It looks like a pleasant day—great for outdoor activities!


👤 You: How does that compare to Melbourne?

🤖 Agent: Right now in Sydney, the weather is clear with no clouds in the sky! The temperature is 18.6°C, feeling a little cooler at 17.9°C. Humidity is at a comfortable 51%, and there’s a gentle breeze with wind speeds of 6.2 m/s. It looks like a pleasant day—great for outdoor activities!


👤 You: How does that compare to Melbourne?

🤖 Agent: Right now in Melbourne, it's partly cloudy (broken clouds) with a warm temperature of 27.3°C, but it actually feels hotter at 30.7°C due to the humidity, which is quite high at 83%. There's also a noticeable breeze with wind speeds around 

## Summary

This notebook demonstrated:

✅ **Environment Configuration** - Loading credentials from `.env` file

✅ **Real Weather API** - Integration with OpenWeatherMap for actual weather data

✅ **AI Agent Creation** - Building a ChatAgent with Azure OpenAI

✅ **Function Calling** - Agent automatically calls weather tools when needed

✅ **Natural Conversation** - Interactive, context-aware weather assistant

✅ **Multi-turn Dialogue** - Maintaining conversation context

### Key Concepts:

1. **Tool Integration** - Weather functions are registered as agent tools
2. **Automatic Invocation** - Agent decides when to call tools based on user queries
3. **Real Data** - Using actual APIs instead of mock data
4. **Environment Management** - Proper credential handling via `.env`

### Try It Yourself:

- Modify the queries in the example cells
- Ask about different cities around the world
- Try complex queries like "Compare weather in 3 cities"
- Ask for travel recommendations based on weather

### Next Steps:

- Add more weather-related tools (forecast, air quality, etc.)
- Create a web UI using the DevUI framework
- Add visualization of weather data
- Integrate with travel planning workflows